# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import xlwings as xw
import numpy as np
import pandas as pd

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
reviews = pd.read_csv("reviews_sample.csv", delimiter=",",index_col="Unnamed: 0")
recipes = pd.read_csv("recipes_sample.csv", delimiter=",")
recipes.drop(['contributor_id','n_steps'],axis=1,inplace=True)

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
excel = 'recipes.xlsx'
recipes_row = recipes.sample(frac = 0.05)
reviews_row = reviews.sample(frac = 0.05)

In [ ]:
with pd.ExcelWriter('recipes.xlsx') as excel:
    recipes_row.to_excel(excel, sheet_name='Рецепты')
    reviews_row.to_excel(excel, sheet_name='Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
recipes_ex = xw.Book('recipes.xlsx')
recipes_sheet = recipes_ex.sheets['Рецепты']

In [ ]:
recipes_sheet.range("H1").value = 'seconds_assign'

In [ ]:
second_assign = (recipes_row["minutes"] * 60).to_numpy()

In [ ]:
recipes_sheet.range("H2:H1501").options(transpose=True).value = second_assign


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
recipes_sheet.range("I1").value = 'seconds_formula'
fml = recipes_sheet.range("I2").formula = f'=D2 * 60'
xw.Range("I2:I1501").formula = fml

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
recipes_sheet.range('H1').api.Font.Bold = True
recipes_sheet.range('I1').api.Font.Bold = True
recipes_sheet.range('H1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
recipes_sheet.range('I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
minutes = recipes_sheet.range("D2:D1501").value
green =  (0, 255, 0)
yellow = (255, 255, 0)
red =  (255, 0, 0)
for i in range(len(minutes)):
    if minutes[i] < 5:
        recipes_sheet.range(f'D{i+2}').color = green
    elif 5 <= minutes[i] <= 10:
        recipes_sheet.range(f'D{i+2}').color = yellow
    else:
        recipes_sheet.range(f'D{i+2}').color = red 
        
        

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
count = len(reviews_row)
recipes_sheet.range('J1').value = 'n_reviews'

In [ ]:
fml = recipes_sheet.range("J2").formula = f'=COUNTIF(Отзывы!$C$2:$C${count},C2)'
xw.Range("J2:J1501").formula = fml

In [ ]:
recipes_sheet.range('J1').api.Font.Bold = True
recipes_sheet.range('J1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 